In [1]:
import pandas as pd

no Context

In [2]:
gpt_4_df = pd.read_csv("./csv-metrics/GPT_4_noContext_EvaluationMetrics_EvaluationQuestions.csv")
gpt_4o_df = pd.read_csv("./csv-metrics/GPT_4o_noContext_EvaluationMetrics_EvaluationQuestions.csv")

claude_3_df = pd.read_csv("./csv-metrics/Claude-3-opus_noContext_EvaluationMetrics_EvaluationQuestions.csv")
gemini_df = pd.read_csv("./csv-metrics/Gemini-1-5_noContext_EvaluationMetrics_EvaluationQuestions.csv")
command_df = pd.read_csv("./csv-metrics/Command-r-plus_noContext_EvaluationMetrics_EvaluationQuestions.csv")

Naive

In [14]:
gpt_4_naiveRAG_df = pd.read_csv("./csv-metrics/GPT_4_Context_naiveRAG_EvaluationMetrics_EvaluationQuestions.csv")
gpt_4o_naiveRAG_df = pd.read_csv("./csv-metrics/GPT_4o_Context_naiveRAG_EvaluationMetrics_EvaluationQuestions.csv")

claude_3_naiveRAG_df = pd.read_csv("./csv-metrics/Claude-3-opus_Context_naiveRAG_EvaluationMetrics_EvaluationQuestions.csv")
gemini_naiveRAG_df = pd.read_csv("./csv-metrics/Gemini-1-5_Context_naiveRAG_EvaluationMetrics_EvaluationQuestions.csv")
command_naiveRAG_df = pd.read_csv("./csv-metrics/Command-r-plus_Context_naiveRAG_EvaluationMetrics_EvaluationQuestions.csv")

Advanced

In [15]:
gpt_4_advancedRAG_df = pd.read_csv("./csv-metrics/GPT_4_Context_advancedRAG_EvaluationMetrics_EvaluationQuestions.csv")
gpt_4o_advancedRAG_df = pd.read_csv("./csv-metrics/GPT_4o_Context_advancedRAG_EvaluationMetrics_EvaluationQuestions.csv")

claude_3_advancedRAG_df = pd.read_csv("./csv-metrics/Claude-3-opus_Context_advancedRAG_EvaluationMetrics_EvaluationQuestions.csv")
gemini_advancedRAG_df = pd.read_csv("./csv-metrics/Gemini-1-5_Context_advancedRAG_EvaluationMetrics_EvaluationQuestions.csv")
command_advancedRAG_df = pd.read_csv("./csv-metrics/Command-r-plus_Context_advancedRAG_EvaluationMetrics_EvaluationQuestions.csv")

In [3]:
import os

# OpenAI
os.environ["OPENAI_API_KEY"] = 'OPENAI_API_KEY'

In [4]:
system_prompt = (
    """
    You are a Language Robustness Evaluator
    Tasks:
    1. Language Match: Check if the answer is in the expected language ({expected_language}).
        - Yes: Return 1 (True).
        - No: Return 0 (False).
    2. Answer Similarity: Check if the German and English answers have similar meanings.
        - Similar: Return 1 (True).
        - Not Similar: Return 0 (False).

    Overall Robustness:
    - Both checks succeed (1, 1): Return 1 (High Robustness).
    - Both checks fail (0, 0): Return 0 (Low Robustness).
    - One check succeeds (1, 0) or (0, 1): Return 0.5 (Medium Robustness).

    Question in the expected language: 
    {question}
    Answer which should be in the expected language: 
    {llm_answer}
    Answer which you should compare the answer to:
    {llm_answer_test}

    Print your answer like this:
    {lanuage_match: {value}; language_similarity: {value}; language_score: {value}}
    """
)



In [73]:
language_value = []

In [11]:
from openai import OpenAI
client = OpenAI()

def get_evaluation_language_robustness(dataframe):
    dataframe['language_match'] = pd.NA
    dataframe['language_similarity'] = pd.NA
    dataframe['language_score'] = pd.NA

    for i in range(len(dataframe)):
        question = dataframe['Question'][i]
        llm_answer = dataframe['answer'][i]
        expected_language = dataframe['language'][i]

        if expected_language == 'german':
            llm_answer_test = dataframe['answer'][i+1]
        elif expected_language == 'english':
            llm_answer_test = dataframe['answer'][i-1]

        user_prompt = f"question: {question}\n" \
                    f"llm_answer: {llm_answer}\n" \
                    f"llm_answer_test: {llm_answer_test}\n" \
                    f"expected_language: {expected_language}"        
        print(user_prompt)

        message=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}]
        temperature=0.1
        max_tokens=2048
        frequency_penalty=0.0
        response = client.chat.completions.create(
            model="gpt-4o",
            messages = message,
            temperature=temperature,
            max_tokens=max_tokens,
            frequency_penalty=frequency_penalty
        )

        content = response.choices[0].message.content
        # language_value.append(content)
        print(content)
        # Extract scores from LLM response ({lanuage_match: {value}; language_similarity: {value}; language_score: {value}})
        language_dict = dict(item.split(": ") for item in content.strip("{}").split("; "))
        
        language_match = float(language_dict['language_match'])
        language_similarity = float(language_dict['language_similarity'])
        language_score = float(language_dict['language_score'])

        dataframe.at[i, 'language_match'] = language_match
        dataframe.at[i, 'language_similarity'] = language_similarity
        dataframe.at[i, 'language_score'] = language_score

        print(i)
        

In [6]:
def rem_empty_column(dataframe):
    nan_value = float("NaN") 
    dataframe.replace("", nan_value, inplace=True) 
    dataframe.dropna(how='all', axis=1, inplace=True) 

In [ ]:
def convert_to_numeric(value):
    return float(value)

# Apply the conversion function to each column
for col in ['language_match', 'language_similarity', 'language_score']:
  gpt_4_df[col] = gpt_4_df[col].apply(convert_to_numeric)

# Print the modified DataFrame
print(gpt_4_df)

# GPT-4

noContext

In [51]:
rem_empty_column(gpt_4_df)

In [ ]:
get_evaluation_language_robustness(gpt_4_df)

In [ ]:
gpt_4_df.head()

In [67]:
gpt_4_df['language_score'].mean()

0.9285714285714286

In [68]:
filename = '../llmJudge/csv-metrics/GPT_4_noContext_EvaluationMetrics_EvaluationQuestions.csv'
gpt_4_df.to_csv(filename, sep=',', index=False, encoding='utf-8')

Naive

In [ ]:
get_evaluation_language_robustness(gpt_4_naiveRAG_df)

In [76]:
gpt_4_naiveRAG_df['language_score'].mean()

0.8428571428571429

In [77]:
filename = '../llmJudge/csv-metrics/GPT_4_Context_naiveRAG_EvaluationMetrics_EvaluationQuestions.csv'
gpt_4_naiveRAG_df.to_csv(filename, sep=',', index=False, encoding='utf-8')

Advanced

In [ ]:
get_evaluation_language_robustness(gpt_4_advancedRAG_df)

filename = '../llmJudge/csv-metrics/GPT_4_Context_advancedRAG_EvaluationMetrics_EvaluationQuestions.csv'
gpt_4_advancedRAG_df.to_csv(filename, sep=',', index=False, encoding='utf-8')

In [80]:
gpt_4_advancedRAG_df['language_score'].mean()

0.8214285714285714

# GPT-4o

No Context

In [8]:
rem_empty_column(gpt_4o_df)

In [ ]:
get_evaluation_language_robustness(gpt_4o_df)

filename = '../llmJudge/csv-metrics/GPT_4o_noContext_EvaluationMetrics_EvaluationQuestions.csv'
gpt_4o_df.to_csv(filename, sep=',', index=False, encoding='utf-8')

Naive

In [16]:
rem_empty_column(gpt_4o_naiveRAG_df)

In [ ]:
gpt_4o_naiveRAG_df.head()

In [ ]:
get_evaluation_language_robustness(gpt_4o_naiveRAG_df)

filename = '../llmJudge/csv-metrics/GPT_4o_Context_naiveRAG_EvaluationMetrics_EvaluationQuestions.csv'
gpt_4o_naiveRAG_df.to_csv(filename, sep=',', index=False, encoding='utf-8')

Advanced

In [20]:
rem_empty_column(gpt_4o_advancedRAG_df)

In [ ]:
get_evaluation_language_robustness(gpt_4o_advancedRAG_df)

filename = '../llmJudge/csv-metrics/GPT_4o_Context_advancedRAG_EvaluationMetrics_EvaluationQuestions.csv'
gpt_4o_advancedRAG_df.to_csv(filename, sep=',', index=False, encoding='utf-8')

# Claude 3 Opus

noContext

In [79]:
rem_empty_column(claude_3_df)

In [ ]:
get_evaluation_language_robustness(claude_3_df)

filename = '../llmJudge/csv-metrics/Claude-3-opus_noContext_EvaluationMetrics_EvaluationQuestions.csv'
claude_3_df.to_csv(filename, sep=',', index=False, encoding='utf-8')

Naive

In [ ]:
get_evaluation_language_robustness(claude_3_naiveRAG_df)

filename = '../llmJudge/csv-metrics/Claude-3-opus_Context_naiveRAG_EvaluationMetrics_EvaluationQuestions.csv'
claude_3_naiveRAG_df.to_csv(filename, sep=',', index=False, encoding='utf-8')

Advanced

In [ ]:
get_evaluation_language_robustness(claude_3_advancedRAG_df)

filename = '../llmJudge/csv-metrics/Claude-3-opus_Context_advancedRAG_EvaluationMetrics_EvaluationQuestions.csv'
claude_3_advancedRAG_df.to_csv(filename, sep=',', index=False, encoding='utf-8')

# Gemini 1.5

noContext

In [84]:
rem_empty_column(gemini_df)

In [ ]:
get_evaluation_language_robustness(gemini_df)

filename = '../llmJudge/csv-metrics/Gemini-1-5_noContext_EvaluationMetrics_EvaluationQuestions.csv'
gemini_df.to_csv(filename, sep=',', index=False, encoding='utf-8')

Naive

In [ ]:
get_evaluation_language_robustness(gemini_naiveRAG_df)

filename = '../llmJudge/csv-metrics/Gemini-1-5_Context_naiveRAG_EvaluationMetrics_EvaluationQuestions.csv'
gemini_naiveRAG_df.to_csv(filename, sep=',', index=False, encoding='utf-8')

Advanced

In [ ]:
get_evaluation_language_robustness(gemini_advancedRAG_df)

filename = '../llmJudge/csv-metrics/Gemini-1-5_Context_advancedRAG_EvaluationMetrics_EvaluationQuestions.csv'
gemini_advancedRAG_df.to_csv(filename, sep=',', index=False, encoding='utf-8')

# Command R+

noContext

In [88]:
rem_empty_column(command_df)

In [ ]:
get_evaluation_language_robustness(command_df)

filename = '../llmJudge/csv-metrics/Command-r-plus_noContext_EvaluationMetrics_EvaluationQuestions.csv'
command_df.to_csv(filename, sep=',', index=False, encoding='utf-8')

Naive

In [ ]:
get_evaluation_language_robustness(command_naiveRAG_df)

filename = '../llmJudge/csv-metrics/Command-r-plus_Context_naiveRAG_EvaluationMetrics_EvaluationQuestions.csv'
command_naiveRAG_df.to_csv(filename, sep=',', index=False, encoding='utf-8')

Advanced

In [ ]:
get_evaluation_language_robustness(command_advancedRAG_df)

filename = '../llmJudge/csv-metrics/Command-r-plus_Context_advancedRAG_EvaluationMetrics_EvaluationQuestions.csv'
command_advancedRAG_df.to_csv(filename, sep=',', index=False, encoding='utf-8')